In [8]:
import pandas as pd
import numpy as np
import math
from math import log, sqrt, exp
from scipy.stats import norm
import matplotlib.pyplot as plt


def get_option_by_s(ST, X, alpha = 0.3):
    """
    这个期权与股票S的关系，分为三种情况
    :param ST: 股票在t=T的时候的价格
    :param X: Strike Price
    :param alpha: 定价因子alpha，在(0, 1)之间
    :return: 返回的是期权的价格
    """
    if alpha * ST >= ST - X >= 0:
        option = ST - X
    if ST - X > alpha * ST:
        option = alpha * ST
    if ST - X < 0:
        option = 0
    return option


def Price_by_BT_2(r, sigma, delta, X, S0, h, alpha = 0.3):
    """
    :param r: 无风险利率
    :param sigma: 历史波动率
    :param delta: 股息dividend
    :param X: Strike Price
    :param S0: 股票在t=0的价格
    :param h: 一次二叉树的时间
    :param alpha: 奇异期权的定价因子
    :return: 返回的是一个dictionary
    """

    u = math.exp((r-delta) * h + sigma*math.sqrt(h))
    d = math.exp((r-delta) * h - sigma*math.sqrt(h))
    p = (math.exp((r-delta) * h) - d)/(u-d)
    q = 1 - p
    uS = u * S0
    dS = d * S0
    uOp = get_option_by_s(uS, X, alpha)
    dOp = get_option_by_s(dS, X, alpha)
    sanjiao = math.exp(-delta*h) * (uOp - dOp)/(S0*(u-d))
    B = math.exp(-r*h) * (u * dOp - d * uOp)/(u-d)
    output_dic = {"Call_Up":uOp,
                  "Call_down":dOp,
                  "Stock_Up":uS,
                  "Stock_Down":dS,
                  "Up_factor":u,
                  "Down_factor":d,
                  "p_star":p,
                  "delta":sanjiao,
                  "B":B}
    return output_dic



def Build_Tree_E(n, r, sigma, delta, X, S0, h, alpha):
    """
    欧式期权的二叉树
    :param n: 进行n次回归
    :param r: 无风险利率
    :param sigma: 历史波动率
    :param delta: dividend的
    :param X: Strike Price
    :param S0_P: 之前股票的价格
    :param h: 每次二叉树的间隔
    :param alpha: 这个是奇异期权的定价中的变量alpha
    :return: 返回是一个嵌套list
    """

    stock_matrix = np.zeros((n+1)*(n+1)).reshape(((n+1), (n+1)))
    option_matrix = np.zeros((n+1)*(n+1)).reshape(((n+1), (n+1)))
    stock_matrix[0, 0] = S0
    for j in range(0, n):
        for i in range(0, j+1):
            S0_P = stock_matrix[i][j]
            if j != i:
                stock_matrix[i, j+1] = Price_by_BT_2(r, sigma, delta, X, S0_P, h, alpha)["Stock_Up"]
            if j == i:
                stock_matrix[i, j+1] = Price_by_BT_2(r, sigma, delta, X, S0_P, h, alpha)["Stock_Up"]
                stock_matrix[i+1, j+1] = Price_by_BT_2(r, sigma, delta, X, S0_P, h, alpha)["Stock_Down"]
    p = Price_by_BT_2(r, sigma, delta, X, S0, h, alpha)["p_star"]
    q = 1 - p
    for i in range(0, n+1):
        option_matrix[i, n] = get_option_by_s(stock_matrix[i, n], X, alpha)
    for j in range(n-1, -1, -1):
        for i in range(0, j+1):
            # print(i, j)
            option_matrix[i, j] = p * option_matrix[i, j+1] + q * option_matrix[i+1, j+1]
    return option_matrix

def Build_Tree_A(n, r, sigma, delta, X, S0, h, alpha):
    """
    美式期权的二叉树
    :param n: 进行n次回归
    :param r: 无风险利率
    :param sigma: 历史波动率
    :param delta: dividend的
    :param X: Strike Price
    :param S0_P: 之前股票的价格
    :param h: 每次二叉树的间隔
    :param alpha: 这个是奇异期权的定价中的变量alpha
    :return: 返回是一个嵌套list
    """

    stock_matrix = np.zeros((n+1)*(n+1)).reshape(((n+1), (n+1)))
    option_matrix = np.zeros((n+1)*(n+1)).reshape(((n+1), (n+1)))
    stock_matrix[0, 0] = S0
    for j in range(0, n):
        for i in range(0, j+1):
            S0_P = stock_matrix[i][j]
            if j != i:
                stock_matrix[i, j+1] = Price_by_BT_2(r, sigma, delta, X, S0_P, h, alpha)["Stock_Up"]
            if j == i:
                stock_matrix[i, j+1] = Price_by_BT_2(r, sigma, delta, X, S0_P, h, alpha)["Stock_Up"]
                stock_matrix[i+1, j+1] = Price_by_BT_2(r, sigma, delta, X, S0_P, h, alpha)["Stock_Down"]

    p = Price_by_BT_2(r, sigma, delta, X, S0, h, alpha)["p_star"]
    q = 1 - p
    for i in range(0, n+1):
        option_matrix[i, n] = get_option_by_s(stock_matrix[i, n], X, alpha)
    for j in range(n-1, -1, -1):
        for i in range(0, j+1):
            # print(i, j)
            price_1 = p * option_matrix[i, j+1] + q * option_matrix[i+1, j+1]
            price_2 = get_option_by_s(stock_matrix[i, j], X, alpha)
            option_matrix[i, j] = max(price_1, price_2)

    return option_matrix

def implied_volatility(n, r, delta, X, S0, h, alpha, Callprice_market):
    Callprice_tree = 0
    err = 0.00001
    left = 0
    right = 1
    sigma = (left+right)/2
    while (abs(Callprice_tree - Callprice_market) >= err):
        Callprice_tree = Build_Tree_E(n, r, sigma, delta, X, S0, h, alpha)[0, 0]

        if Callprice_tree < Callprice_market:
            left = sigma
            sigma = (sigma + right)/2
        elif Callprice_tree >= Callprice_market:
            right = sigma
            sigma = (sigma + left)/2
#         print(abs(Callprice_tree-Callprice_market), Callprice_tree, sigma)
    return sigma


### 3.3 隐含波动率

In [4]:
n = 100
X = 3.2
S0 = 3.543
Callprice_market = 0.352639
T = 1/12
h = T/n
sigma_h = 0.5375968992057021
delta = 0.05/3.5
delta = 0
alpha = 0.2
r = 0.029905


sigma_i = implied_volatility(n, r, delta, X, S0, h, alpha, Callprice_market)

0.016276426503731012 0.368915426503731 0.25
0.0027372418982565816 0.35537624189825656 0.125
0.0007469450283192347 0.35189205497168075 0.1875
0.0002975319934452325 0.3529365319934452 0.15625
0.00043907860933273257 0.35219992139066725 0.171875
0.00012333761261457132 0.3525156623873854 0.1796875
1.7819760625759073e-05 0.35265681976062574 0.17578125
5.2758590477652856e-05 0.3525862414095223 0.177734375
1.746933416763019e-05 0.35262153066583235 0.1787109375
1.7523302664468332e-07 0.3526391752330266 0.17822265625
隐含波动率 0.17822265625


In [5]:
print("隐含波动率", sigma_i)

隐含波动率 0.17822265625


### 3.4. 隐含波动率曲线

In [12]:
X = np.linspace(3.15, 3.25, 100)
X

array([3.15      , 3.1510101 , 3.1520202 , 3.1530303 , 3.1540404 ,
       3.15505051, 3.15606061, 3.15707071, 3.15808081, 3.15909091,
       3.16010101, 3.16111111, 3.16212121, 3.16313131, 3.16414141,
       3.16515152, 3.16616162, 3.16717172, 3.16818182, 3.16919192,
       3.17020202, 3.17121212, 3.17222222, 3.17323232, 3.17424242,
       3.17525253, 3.17626263, 3.17727273, 3.17828283, 3.17929293,
       3.18030303, 3.18131313, 3.18232323, 3.18333333, 3.18434343,
       3.18535354, 3.18636364, 3.18737374, 3.18838384, 3.18939394,
       3.19040404, 3.19141414, 3.19242424, 3.19343434, 3.19444444,
       3.19545455, 3.19646465, 3.19747475, 3.19848485, 3.19949495,
       3.20050505, 3.20151515, 3.20252525, 3.20353535, 3.20454545,
       3.20555556, 3.20656566, 3.20757576, 3.20858586, 3.20959596,
       3.21060606, 3.21161616, 3.21262626, 3.21363636, 3.21464646,
       3.21565657, 3.21666667, 3.21767677, 3.21868687, 3.21969697,
       3.22070707, 3.22171717, 3.22272727, 3.22373737, 3.22474

In [13]:
volatility_smile = []
for i in X:
    sigma_i = implied_volatility(n, r, delta, i, S0, h, alpha, Callprice_market)
    volatility_smile.append(sigma_i)

    
plt.plot(X, volatility_smile)

ZeroDivisionError: float division by zero